In [2]:
from AOC_utils import get_day
import numpy as np

day = 18
input_data = get_day(day)

# print out first few lines to get a feel for the data
print(len(input_data))
input_data[:7]

Day 18 input already downloaded
3450


['40,65', '17,1', '34,45', '31,51', '29,43', '25,9', '14,27']

In [3]:
# part 1
example = '''
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
'''.split('\n')[1:-1]

In [29]:
def solve_day(data, sim=1024, grid_size=70):
    part1 = 0
    part2 = 0

    bits = np.array([line.split(',') for line in data], dtype=int)
    # print(bits)

    def get_shortest_path(n_bytes=sim):
        grid = np.zeros((grid_size+1, grid_size+1), dtype=int)
        grid[bits[:n_bytes, 1], bits[:n_bytes, 0]] = 1
        # for line in grid:
        #     print(''.join(['#' if x else '.' for x in line]))

        start = [0, 0]
        end = np.array([grid_size, grid_size])

        directions = np.array([[0,1], [0,-1], [1,0], [-1,0]])
        
        np_paths = np.array([start])
        visited = np.full((grid_size+1, grid_size+1), fill_value=np.inf)
        visited[tuple(start)] = 0

        np_directions = np.array(directions).transpose(1,0)[None,...]

        bs = 32

        while len(np_paths) > 0:
            if visited[tuple(end)] != np.inf:
                break
            
            np_paths = np_paths[np.argsort(np.sum(np_paths - end, axis=-1))[::-1]]

            lengths = visited[np_paths[:bs,0], np_paths[:bs,1]]
            new_paths = np_paths[:bs][...,None] + np_directions
            new_paths = new_paths.transpose(0,2,1).reshape((-1,2))
            lengths = np.repeat(lengths,4)
            boo = ~np.bitwise_or(np.any(new_paths < 0, axis=-1), 
                                    np.any(new_paths > grid_size, axis=-1))
            new_paths = new_paths[boo]; lengths = lengths[boo]
            boo = grid[new_paths[:,0], new_paths[:,1]] == 0
            new_paths = new_paths[boo]; lengths = lengths[boo]
            boo = lengths+1 < visited[new_paths[:,0], new_paths[:,1]]
            new_paths = new_paths[boo]
            lengths = lengths[boo] + 1

            visited[new_paths[:,0], new_paths[:,1]] = lengths

            np_paths = np.concatenate([np_paths[bs:], new_paths], axis=0)

        return visited[tuple(end)]

    part1 = int(get_shortest_path(sim))
    print("part 1:", part1)


    for i in range(sim, len(bits)):
        if get_shortest_path(i) == np.inf:
            part2 = str(bits[i-1][0]) + ',' + str(bits[i-1][1])
            break
        # if i % 100 == 0:
        #     print(i/len(bits))

    print("part 2:", part2)

solve_day(example, sim=12, grid_size=6)

part 1: 22
part 2: 6,1


In [30]:
solve_day(input_data)

part 1: 344
part 2: 46,18
